csv 파일을 생성하는 코드
학습할 때 이미지 경로를 직접 설정하지 않고, csv 파일에서 경로를 불러옴
base_directory: 등급 폴더들이 있는 폴더 경로
label_directory: 등급별 라벨 엑셀 파일들이 있는 폴더 경로

csv 형식

No, grade, Marbling, Color, Texture, Surface_Moisture, Total, image_path

ex) 123,등심1++,8.0,6.0,8.0,4.0,8.0,./meat_dataset/등심1++/등심1++_000353.jpg


경로 설정 방법

이미지 경로 설정 방법:
 - base_directory: 등급 폴더가 있는 폴더의 경로 입력
 - image_directories: 각 등급에 맞는 base_directory 안의 등급 폴더 경로 입력
   ex) image_directories = {'등급': os.path.join(base_directory, '등급 파일 경로')}

라벨 경로 설정 방법:
 - label_directory: 엑셀 라벨 파일들이 있는 폴더 경로
 - excel_files: 각 등급에 맞는 라벨 엑셀 파일이름 입력


이미지 이름 형식: 등심1++_000001.jpg, 등심2_001234.jpg (숫자는 6자리로 패딩)

csv에 저장되는 경로 예시: ./dataset/meat_dataset/등심1++_000738.jpg

In [1]:
import os
import pandas as pd
import ast

In [2]:
def find_header_row(file_path):
    required_columns = ['No', '등급', 'Marbling(마블링정도)', 'Color(색깔)', 'Texture(조직감)', 'Surface Moisture(표면육즙)', 'Total(기호도)']
    
    # 엑셀 파일을 열고 각 행을 확인
    for i in range(20):
        df = pd.read_excel(file_path, header=i, nrows=1)
        if all(col in df.columns for col in required_columns):
            return i
    
    raise ValueError(f"Required columns not found in the first 20 rows of {file_path}")

In [3]:
# 제외할 이미지 목록 읽기 함수
def read_exclude_list(file_path):
    exclude_images = set()
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                try:
                    # 문자열을 리스트로 변환
                    image_list = ast.literal_eval(line.strip())
                    # 리스트의 각 이미지를 set에 추가
                    exclude_images.update(image_list)
                except:
                    # 라인을 파싱할 수 없는 경우 무시
                    continue
    return exclude_images

In [4]:
def process_data(base_directory, image_directories, excel_files, output_file):
    exclude_file_path1 = '차이_4이상.txt'
    exclude_file_path2 = 'nan_값_포함.txt'
    exclude_images = read_exclude_list(exclude_file_path1).union(read_exclude_list(exclude_file_path2))
    
    print(len(exclude_images))
    print(next(iter(exclude_images)))

    dataframes = []
    for grade, file_path in excel_files.items():
        try:
            header_row = find_header_row(file_path)
            df = pd.read_excel(file_path, header=header_row)
            
            df = df.dropna(subset=['No', '등급'])
            df = df.dropna(subset=['Marbling(마블링정도)', 'Color(색깔)', 'Texture(조직감)', 'Surface Moisture(표면육즙)', 'Total(기호도)'])
            
            columns_to_check = ['Marbling(마블링정도)', 'Color(색깔)', 'Texture(조직감)', 'Surface Moisture(표면육즙)', 'Total(기호도)']
            for column in columns_to_check:
                df = df[pd.to_numeric(df[column], errors='coerce').notnull()]
#                 df[column] = df[column].astype(float) * 2
            
            # 등심1에 대해서만 No 값 조정
            if grade == '등심1':
                df['No'] = df['No'].apply(lambda x: x - 1 if x > 103 else x)

            # '등급' 열을 기준으로 이미지 경로 설정
            df['image_path'] = df.apply(lambda row: os.path.join(image_directories[grade], f"{grade}_{int(row['No']):06d}.jpg"), axis=1)
            
            # 존재하는 이미지 파일만 필터링하고 제외 목록에 없는 이미지만 선택
            df = df[df['image_path'].apply(lambda x: os.path.exists(x) and os.path.basename(x) not in exclude_images)]
            
            # 유효한 파일 경로가 몇 개인지 로그 출력
            max_no = df['No'].max()
            print(f"Filtered valid image paths for {grade}: {len(df)} / {max_no}")
            
            dataframes.append(df)

        except ValueError as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue

    # 모든 데이터프레임 병합
    all_data = pd.concat(dataframes, ignore_index=True)
    all_data.columns = all_data.columns.str.replace(r'\(.*\)', '', regex=True).str.strip()
    all_data.columns = all_data.columns.str.replace(' ', '_')
    
    # 필요한 열만 선택 (Total 포함)
    all_data = all_data[['No', '등급', 'Marbling', 'Color', 'Texture', 'Surface_Moisture', 'Total', 'image_path']]
    
    # '등급' 열의 모든 값 앞에 '등심' 추가
    all_data['등급'] = '등심' + all_data['등급'].astype(str)

    # '등급' 열 이름을 'grade'로 변경
    all_data = all_data.rename(columns={'등급': 'grade'})
    
    # image_path에서 '../' 제거
    all_data['image_path'] = all_data['image_path'].str.replace('../', '', 1)

    # CSV 파일로 저장
    all_data.to_csv(os.path.join('../dataset/', output_file), index=False)
    print(f"Data saved to {os.path.join(base_directory, output_file)}")

    return all_data

In [5]:
base_directory = "../dataset/meat_dataset/"
label_directory = "../dataset/meat_dataset/labels"
output_file = "1to5.csv"

# 이미지 파일 경로 설정
image_directories = {
    '등심1++': os.path.join(base_directory, '등심1++'),
    '등심1+': os.path.join(base_directory, '등심1+'),
    '등심1': os.path.join(base_directory, '등심1'),
    '등심2': os.path.join(base_directory, '등심2'),
    '등심3': os.path.join(base_directory, '등심3')
}
# 엑셀 파일 경로 설정
excel_files = {
    '등심1++': os.path.join(label_directory, 'label_1++.xlsx'),
    '등심1+': os.path.join(label_directory, 'label_1+.xlsx'),
    '등심1': os.path.join(label_directory, 'label_1.xlsx'),
    '등심2': os.path.join(label_directory, 'label_2.xlsx'),
    '등심3': os.path.join(label_directory, 'label_3.xlsx')
}

In [6]:
processed_data = process_data(base_directory, image_directories, excel_files, output_file)

1604
등심2_001147.jpg
Filtered valid image paths for 등심1++: 1745 / 2326
Filtered valid image paths for 등심1+: 1554 / 2122
Filtered valid image paths for 등심1: 1437 / 1921
Filtered valid image paths for 등심2: 995 / 1252
Filtered valid image paths for 등심3: 434 / 539
Data saved to ../dataset/meat_dataset/1to5.csv


/tmp/ipykernel_152209/4191139284.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  all_data['image_path'] = all_data['image_path'].str.replace('../', '', 1)
